In [ ]:
import os
import tqdm
import geojson
import rasterio
import firedrake
import icepack, icepack.plot
import plumes

### Geometry

First, we'll load in a GeoJSON file describing the outline of Pine Island Ice Shelf.
This outline was hand-digitized from the various input data sets we'll use in a GIS.

In [ ]:
outline_filename = 'pine-island-shelf.geojson'
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

Next we'll use a few utility functions from icepack to turn this outline into the input format for the mesh generator [gmsh](https://www.gmsh.info).
We can then generate an unstructured triangular mesh of the domain and load in that mesh.

In [ ]:
geometry = icepack.meshing.collection_to_geo(outline)
with open('pine-island-shelf.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o pine-island-shelf.msh pine-island-shelf.geo

In [ ]:
mesh = firedrake.Mesh('pine-island-shelf.msh')

The colors correspond to the numeric IDs of each boundary segment (note the legend in the corner).
We need a way of identifying different boundary conditions in the ice shelf flow model in order to determine where ice is flowing in from and where the terminus is.

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.legend();

### Input data

Next we have to load in some observational data for the ice shelf draft.
The demos for icepack include a module to fetch the most common observational data sets.
We'll use BedMachine Antarctica because they've already done the hard work of things like firn and geoid corrections to ice shelf thickness.
This routine will download the BedMachine dataset from NSIDC.
If this is your first time running this notebook, you'll be prompted for your EarthData username and password.

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()

To get the ice shelf draft, we'll first get the surface elevation and thickness of the ice shelf.

In [ ]:
surface_grid = rasterio.open('netcdf:' + bedmachine_filename + ':surface', 'r')
thickness_grid = rasterio.open('netcdf:' + bedmachine_filename + ':thickness', 'r')

We'll represent the ice shelf draft using continuous, piecewise quadratic basis functions.

In [ ]:
Z = firedrake.FunctionSpace(mesh, family='CG', degree=2)

Next we'll interpolate the gridded data to our finite element space.

In [ ]:
surface = icepack.interpolate(surface_grid, Z)
thickness = icepack.interpolate(thickness_grid, Z)

Finally we can get the ice shelf draft as the difference of the surface and the thickness.

In [ ]:
z_obs = firedrake.interpolate(surface - thickness, Z)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z_obs, 40, axes=axes)
fig.colorbar(contours);

In the interest of keeping our numerical solver from losing its bloody mind, we'll smooth over the ice shelf.
The spin-up of the plume will proceed in two stages.
First, we'll use a very smooth ice shelf draft.
Most of the interesting features, like sub-ice shelf channels, will have been diffused out.
But this will make it easier to evolve the plume towards a sane steady state than if we had used the real data as-is.
In the next phase, we'll slowly morph the ice shelf draft towards a value with a much lower smoothing length in order to recapture more of the features of the real data.

In [ ]:
from firedrake import inner, grad, dx, ds, Constant
z = firedrake.Function(Z)
α = firedrake.Constant(5e3)

parameters = {
    'solver_parameters': {
        'ksp_type': 'preonly',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps'
    }
}

J = 0.5 * ((z - z_obs)**2 + α**2 * inner(grad(z), grad(z))) * dx
F = firedrake.derivative(J, z)
firedrake.solve(F == 0, z, **parameters)

This is enough smoothing to keep the plume solver from exploding, but not so much as to wipe out features like sub-ice shelf channels.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(z, 40, axes=axes)
fig.colorbar(contours);

Some amount of smoothing is also usually necessary for the ice flow model too.
Newer remote sensing platforms like ICESat-2 are sophisticated enough to resolve individual crevasses, introducing sharp breaks in the ice thickness.
The gradient of the ice thickness is one of the sources of the ice flow model, so these features, which are of too small a scale to really influence the flow by themselves, end up breaking the numerics.

### Initial state

Next, we need to come up with some vaguely sane initial state of the plume.
The model can quickly explode if we initialize it with a weird initial state that has large transients, so we need to find something vaguely reasonable.
Roughly speaking, the fluid velocity should align with the gradient of the ice shelf bottom.

In [ ]:
W = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
grad_z = firedrake.project(grad(z), W)

To make the initial velocity field, we'll solve an optimization problem.
The objective functional will favor a velocity field that is:

1. smoothly varying
2. non-divergent
3. aligns with the ice shelf draft gradient whenever it points into the domain

Putting all of these criteria together, we get the objective 

$$J = \frac{\alpha^2}{2}\int_\Omega\left\{|\nabla u|^2 + (\nabla\cdot u)^2\right\}dx + \frac{\alpha}{2}\int_\Gamma |u - v|^2ds$$

where $\Gamma$ is everything except the ice shelf terminus, $v$ is the inflow vector

$$v = \max\{0, \nu\cdot\nabla z_b\}\nu,$$

and $\nu$ the unit inward-pointing normal to the domain.

In [ ]:
from firedrake import div
v = firedrake.Function(W)

n = -firedrake.FacetNormal(mesh)
v_n = firedrake.max_value(inner(grad_z, n), 0) * n

terminus_ids = (2, 3, 4)
J = (
    0.5 * α**2 * (inner(grad(v), grad(v)) + div(v)**2) * dx +
    0.5 * α * inner(v - v_n, v - v_n) * ds(terminus_ids)
)
F = firedrake.derivative(J, v)
firedrake.solve(F == 0, v, **parameters)

As a sanity check we can make a stream plot of the initial value of the plume velocity.
The streamlines get thicker towards the end, so this vector field actually is going from the inflow boundary to the terminus as we had hoped.

In [ ]:
fig, axes = icepack.plot.subplots()
firedrake.triplot(
    mesh, axes=axes,
    boundary_kw={'color': 'k'},
    interior_kw={'linewidth': 0.05}
)
streamlines = firedrake.streamplot(v, resolution=2e3, seed=1, axes=axes)
fig.colorbar(streamlines);

The finite element space we used to calculate $v$ is not what we'll want to use to advect it through the ice shelf.
For solving the (hyperbolic) PDEs describing flow in the plume, we'll instead want to use a discontinuous Galerkin basis.
To keep things simple we'll use piecewise constant basis functions in each triangle.
This basis makes our method is identical to a first-order finite volume method.
To get a higher-order discretization, we could use basis functions with a higher polynomial degree.
We would then need to also use a more accurate timestepping scheme and some kind of flux-limiting scheme.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='DG', degree=0)
V = firedrake.FunctionSpace(mesh, family='RT', degree=1)

In [ ]:
u_0 = firedrake.project(v, V)

Next, we'll need some initial values for the plume thickness, temperature, and salinity.
We'll use some very rough initial guesses for these based on the values we found in the synthetic demo.

In [ ]:
D_0 = firedrake.Function(Q)
S_0 = firedrake.Function(Q)
T_0 = firedrake.Function(Q)

D_0.project(Constant(0.01))
S_0.project(Constant(34.))
T_0.project(Constant(-2.))

Similarly, we'll use constant guesses for the background ocean salinity and temperature.

In [ ]:
T_hssw = Constant(-1.91)
S_hssw = Constant(34.65)

Now we can start setting up the plume model as in the synthetic demo.

In [ ]:
fields = {
    'thickness': D_0.copy(deepcopy=True),
    'velocity': u_0.copy(deepcopy=True),
    'temperature': T_0.copy(deepcopy=True),
    'salinity': S_0.copy(deepcopy=True)
}

inflow = {
    'thickness_inflow': D_0,
    'velocity_inflow': u_0,
    'temperature_inflow': T_0,
    'salinity_inflow': S_0
}

inputs = {
    'ice_shelf_base': z,
    'salinity_ambient': S_hssw,
    'temperature_ambient': T_hssw
}

We'll start by just spinning up the mass, salinity, and temperature of the plume, holding the velocity fixed.
The reason we're doing this is to get a critical thickness of the plume before spinning up the velocity.
If we don't do this carefully, we can easily end up with a negative plume thickness.

In [ ]:
from plumes import Component

model = plumes.PlumeModel()
components = Component.Mass | Component.Salt | Component.Heat
solver = plumes.PlumeSolver(
    model, components,
    **fields, **inflow, **inputs
)

Before we can begin, we need to pick a timestep that satisfies the Courant-Friedrichs-Lewy condition.

In [ ]:
import numpy as np
#U = np.sqrt(np.sum(u_0.dat.data_ro[:]**2, axis=1)).max()
δx = mesh.cell_sizes.dat.data_ro[:].min()
#print(δx / U)
print(δx)

This is a very long timestep so if we instead use a value of, say 120s we'll be fine.

In [ ]:
timestep = 120.
num_days = 1.
final_time = num_days * 24 * 60 * 60
num_steps = int(final_time / timestep)
print(f'Number of steps: {num_steps}')
dt = final_time / num_steps

In [ ]:
import tqdm
pbar = tqdm.trange(num_steps)
for step in pbar:
    solver.step(dt)

    if step % 50 == 0:
        D = solver.fields['thickness']
        Dmin = D.dat.data_ro[:].min()
        Dmax = D.dat.data_ro[:].max()
        
        pbar.set_description(f"thickness min/max: ({Dmin:4.2f}, {Dmax:4.2f})")

In [ ]:
fig, axes = icepack.plot.subplots()
D = solver.fields['thickness']
triangles = icepack.plot.tripcolor(D, axes=axes)
fig.colorbar(triangles);

Now we can start spinning up the plume velocity.

In [ ]:
def run_stage(fields_initial, name, num_days, timestep):
    fields = {
        name: field.copy(deepcopy=True)
        for name, field in fields_initial.items()
    }
    
    if os.path.exists(name + '.h5'):
        with firedrake.DumbCheckpoint(name, mode=firedrake.FILE_READ) as chk:
            for name in ('thickness', 'temperature', 'salinity', 'velocity'):
                chk.load(fields[name], name=name)
                
        return fields

    components = Component.Mass | Component.Salt | Component.Heat | Component.Momentum
    solver = plumes.PlumeSolver(
        model, components,
        **fields, **inflow, **inputs
    )

    final_time = num_days * 24 * 60 * 60
    num_steps = int(final_time / timestep)
    dt = final_time / num_steps

    pbar = tqdm.trange(num_steps)
    for step in pbar:
        solver.step(dt)

        if step % 50 == 0:
            D = solver.fields['thickness']
            Dmin = D.dat.data_ro[:].min()
            Dmax = D.dat.data_ro[:].max()

            pbar.set_description(
                f"thickness min/max: ({Dmin:4.2f}, {Dmax:4.2f})"
            )

    with firedrake.DumbCheckpoint(name, mode=firedrake.FILE_CREATE) as chk:
        for name in ('thickness', 'temperature', 'salinity', 'velocity'):
            chk.store(solver.fields[name], name=name)
            fields[name].assign(solver.fields[name])
            
    return fields

In [ ]:
fields_initial = {
    name: field.copy(deepcopy=True)
    for name, field in solver.fields.items()
}

In [ ]:
fields_stage_1 = run_stage(fields_initial, 'stage-1', num_days=0.5, timestep=5)

In [ ]:
fields_stage_2 = run_stage(fields_stage_1, 'stage-2', num_days=0.5, timestep=2.5)

In [ ]:
fields_stage_3 = run_stage(fields_stage_2, 'stage-3', num_days=1., timestep=1.)

In [ ]:
fields_stage_4 = run_stage(fields_stage_3, 'stage-4', num_days=1., timestep=2.)

In [ ]:
fields_stage_5 = run_stage(fields_stage_4, 'stage-5', num_days=1., timestep=5.)